In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

DataFrame = pd.read_csv("../0703/all_lat.csv")
common_used_numerals_tmp = {'零': 0, '一': 1, '二': 2, '两': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9,
                            '十': 10, '百': 100, '千': 1000, '万': 10000, '亿': 100000000}
common_used_numerals = {}
for key in common_used_numerals_tmp:
    common_used_numerals[key] = common_used_numerals_tmp[key]


def chinese2digits(uchars_chinese):
    total = 0
    r = 1  # For digits
    for i in range(len(uchars_chinese) - 1, -1, -1):
        val = common_used_numerals.get(uchars_chinese[i])
        if val >= 10 and i == 0:  #for 十三、十八...
            if val > r:
                r = val
                total = total + val
            else:
                r = r * val
        elif val >= 10:
            if val > r:
                r = val
            else:
                r = r * val
        else:
            total = total + r * val
    return total
 
 
num_str_start_symbol = ['一', '二', '两', '三', '四', '五', '六', '七', '八', '九','十']
more_num_str_symbol = ['零', '一', '二', '两', '三', '四', '五', '六', '七', '八', '九', '十', '百', '千', '萬', '億']
#中轉數
def changeChineseNumToArab(oriStr):
    lenStr = len(oriStr);
    aProStr = ''
    if lenStr == 0:
        return aProStr;

    hasNumStart = False;
    numberStr = ''
    for idx in range(lenStr):
        if oriStr[idx] in num_str_start_symbol:
            if not hasNumStart:
                hasNumStart = True;
 
            numberStr += oriStr[idx]
        else:
            if hasNumStart:
                if oriStr[idx] in more_num_str_symbol:
                    numberStr += oriStr[idx]
                    continue
                else:
                    numResult = str(chinese2digits(numberStr))
                    numberStr = ''
                    hasNumStart = False;
                    aProStr += numResult
 
            aProStr += oriStr[idx]
            pass
 
    if len(numberStr) > 0:
        resultNum = chinese2digits(numberStr)
        aProStr += str(resultNum)

    return aProStr

num=['零','一','二','三','四','五','六','七','八','九']
 
k=['零','十','百','千','萬','十','百']
 
def rankid():
    rank=[]
    for i in range(99):
        a=tstr(i)
        rank.append(a)
    return rank
 
def turn(x,y):
    if y>= 1:
        a=x//pow(10,y)
        b=x%pow(10,y)
        c=num[a]+k[y]
        if y>4 and b<pow(10,4):
            c+=k[4]
        if (len(str(x))-len(str(b))) >= 2 and b != 0:
            c+=k[0]
    else:
        a=x
        b=0
        c=num[a]
 
    return (c,b,)

#數轉中
def tstr(x):
    c=turn(x,(len(str(x))-1))
    a=c[0]
    b=c[1]
    while b != 0:
        a+=turn(b,(len(str(b))-1))[0]
        b=turn(b,(len(str(b))-1))[1]
    if x >= 10 and x < 20:
        a = a.replace("一","")
    if x == 11:
        a = "十一"
    return a
#%%
#for land data convert
def floor_Conv(s):
    DataFrame[s] = DataFrame[s].str.replace("地下層","0")
    DataFrame[s] = DataFrame[s].str.replace("層","")
    DataFrame[s] = DataFrame[s].str.replace("夾","夾層")
    DataFrame[s] = DataFrame[s].str.replace("地下","-")
    for i in range(100,0,-1):
        DataFrame[s] = DataFrame[s].str.replace(tstr(i),changeChineseNumToArab(tstr(i)))
    DataFrame[s] = DataFrame[s].str.replace("101","11")
#print(tstr(16)) OUTPUT : 十六
#print(changeChineseNumToArab("二十一")) OUTPUT : 21
floor_Conv("移轉層次")
floor_Conv("總樓層數")
DataFrame['移轉層次'].fillna("NA",inplace = True)
DataFrame['總樓層數'].fillna("NA",inplace = True)

#%%
#Store special
allforNone = []
for i in range(DataFrame.shape[0]):
    if DataFrame['移轉層次'][i] != "NA" and DataFrame['移轉層次'][i] != "全" and DataFrame['總樓層數'][i] == "NA":
#         print(i)
        a = re.findall(r'\d+',DataFrame['建物型態'][i])
        DataFrame['總樓層數'][i] = a[0]
        
DataFrame_House = DataFrame.loc[DataFrame['建物型態'].str.contains('透天厝')]
DataFrame_House = DataFrame_House.reset_index(drop=True)
DataFrame_House['移轉層次'].fillna("NA",inplace = True)
DataFrame_House['總樓層數'].fillna("NA",inplace = True)
DataFrame_House['總樓層數'] = DataFrame_House['總樓層數'].str.replace("NA",'0')
DataFrame_House = DataFrame_House.astype({"總樓層數": int})

#get mode
counts = np.bincount(DataFrame_House['總樓層數'])
a = np.argmax(counts)
#%%
for i in range(0, DataFrame.shape[0]):
    string = ""
    if DataFrame['移轉層次'][i] == "全" and DataFrame['總樓層數'][i] == "NA":
        DataFrame['總樓層數'][i] = str(a)
    if DataFrame['移轉層次'][i] == "全" and DataFrame['總樓層數'][i] != "NA":
        times = int(DataFrame['總樓層數'][i])
        for j in range(times):
            string += str(j+1)
            if j != times - 1:
                string += '，'
        DataFrame['移轉層次'][i] = string
    #地下分開
    #float ()
#%%
#For memorize DataFrame original data
Original_Data = DataFrame
#%%
#Reset DataFrame
DataFrame = Original_Data
#%%
DataFrame['總樓層數'] = DataFrame['總樓層數'].str.replace("NA",'0')
Data_Test = DataFrame
Data_Test= Data_Test.astype({"總樓層數": float})
num = []
char = []
negative_num = []
for i in range(DataFrame.shape[0]):
    num_string = ""
    char_string = ""
    Negative_string = ""
    num_sum = 0
    Negative_sum = 0
    x = str(DataFrame['移轉層次'][i]).split('，')
    for j in range(len(x)):
        if x[j].isdigit():
            num_sum += int(x[j])
        else:
            if '-' in x[j]:
                Negative_sum += int(x[j])
            else:
                char_string += x[j]
                if j != len(x)-1:
                    char_string += "，"
    if (DataFrame['總樓層數'][i] == '0'):
        Total_num = 0.0
    else:
        Total_num = num_sum/float(DataFrame['總樓層數'][i])
        if Negative_sum == 0:
            Total_negative_num = Negative_sum / float(DataFrame['總樓層數'][i])
        else:
            Total_negative_num = -(Negative_sum / float(DataFrame['總樓層數'][i]))
    num.append(Total_num)
    char.append(char_string)
    negative_num.append(Total_negative_num)
DataFrame['移轉層次(正數)'] = num
DataFrame['移轉層次(非正數)'] = negative_num
DataFrame['移轉層次(非純數)'] = char

c:\users\wade\.virtualenvs\19'_summer_vacation-ib8vnh7u\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


8286
14050
14051


c:\users\wade\.virtualenvs\19'_summer_vacation-ib8vnh7u\lib\site-packages\ipykernel_launcher.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


24603
28524
36706
38180
44349
44640
50522
58432
59018


c:\users\wade\.virtualenvs\19'_summer_vacation-ib8vnh7u\lib\site-packages\ipykernel_launcher.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\wade\.virtualenvs\19'_summer_vacation-ib8vnh7u\lib\site-packages\ipykernel_launcher.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [2]:
DataFrame.head()

,鄉鎮市區,交易標的,土地區段位置/建物區段門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,...,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號,lat,lng,移轉層次(正數),移轉層次(非正數),移轉層次(非純數)
0,板橋區,房地(土地+建物),新北市板橋區華東街271~300號,8.80,住,NaN,NaN,1060621,土地2建物1車位0,19,...,NaN,0.00,0,本案尚未成立管理委員會.,RPUOMLMJQHMFFAF48CA,25.001936,121.452653,0.791667,0.0,
1,板橋區,房地(土地+建物)+車位,新北市板橋區華東街271~300號,18.48,住,NaN,NaN,1060725,土地2建物1車位1,18,...,坡道平面,26.73,2600000,本案尚未成立管理委員會.,RPXOMLMJQHMFFAF78CA,25.001936,121.452653,0.750000,0.0,
2,土城區,房地(土地+建物),新北市土城區永豐路121~150號,17.71,住,NaN,NaN,1060617,土地6建物1車位0,9,...,NaN,0.00,0,NaN,RPORMLMJQHMFFAF18CA,24.984078,121.466060,0.642857,0.0,
3,板橋區,房地(土地+建物)+車位,新北市板橋區新崑路91~120號,26.23,住,NaN,NaN,1060630,土地1建物1車位1,8,...,坡道平面,24.71,0,NaN,RPQNMLNJQHMFFAF08CA,24.999177,121.432317,0.615385,0.0,
4,土城區,房地(土地+建物),新北市土城區延和路76巷1~30號,22.47,住,NaN,NaN,1060620,土地1建物1車位0,1,...,NaN,0.00,0,NaN,RPSOMLNJQHMFFAF38CA,24.989084,121.466463,0.200000,0.0,


### 輸出成樓層特徵csv

In [4]:
DataFrame.to_csv('floor_feature_all.csv', index=False, encoding='utf_8_sig')

In [5]:
DataFrame[['移轉層次(正數)', '移轉層次(非正數)', '移轉層次(非純數)']] .to_csv('floor_feature.csv', index=False, encoding='utf_8_sig')